<a href="https://colab.research.google.com/github/abhisinghh/RAG_over_doc/blob/main/dense_rag_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install langchain-community
!pip install langchain_experimental
!pip install langchain_openai
!pip install pypdf
!pip install openai
# Install Gradio
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.8 MB/s eta 0:00:00


In [ ]:
!git clone   https://github.com/abhisinghh/RAG_over_doc.git

Cloning into 'RAG_over_doc'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 18 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 59.65 KiB | 2.29 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [ ]:
%cd RAG_over_doc

/content/RAG_over_doc


In [ ]:
!pwd

/content/RAG_over_doc


In [ ]:
import yaml
import os
import gradio as gr
import json
from langchain.document_loaders import PyPDFLoader
import openai
import numpy as np

In [ ]:
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

os.environ["OPENAI_API_KEY"] =  config['openai_api_key']

In [ ]:
from agentic_rag import *
from chunker import chunk_documents


In [ ]:
def process_files(pdf_file, txt_file,status_update):
    json_file_path = "output.json"
    jsonified_responses = []

    if pdf_file is None:
        return None, "No PDF uploaded."

    if txt_file is None:
        return None, "No text file with questions uploaded."

    loader = PyPDFLoader(pdf_file.name)
    documents = loader.load()
    ## chunking he documents
    df_chunks = chunk_documents(documents)

    print(df_chunks.shape)
    df_chunks['embedding'] = df_chunks['Chunks'].apply(get_embedding)

    with open(txt_file.name, "r") as f:
        questions = [question.strip() for question in f.readlines()]

    for query in questions:
        top_k_df = find_most_similar(query, df_chunks)
        context = "\n".join(top_k_df['Chunks'])
        messages = [
            {"role": "system", "content": "You are a helpful assistant that answers questions based on the given context. NOTE: If you do not know the answer just return 'Data not available'"},
            {"role": "user", "content": f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"}
        ]
        response = openai.chat.completions.create(
            model="gpt-4o-mini",  # Ensure this is the correct model name
            messages=messages,
            temperature=0
        )
        jsonified_responses.append({"query": query, "answer": response.choices[0].message.content})

    output_json = json.dumps(jsonified_responses, indent=4)
    with open(json_file_path, "w") as f:
        f.write(output_json)

    return json_file_path, f"Processed {len(questions)} questions. Results saved to {json_file_path}"

# Create the Gradio interface
iface = gr.Interface(
    fn=process_files,
    inputs=[
        gr.File(label="Upload PDF Document"),
        gr.File(label="Upload Text File with Questions")
    ],

    outputs=[
        gr.File(label="Download JSON Results"),
        "text"
    ],
    title="Upload PDF and Questions",
    description="Upload a PDF document and a text file containing a list of questions. The results will be available for download as a JSON file."
)

# Launch the interface
iface.launch()

/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1002: UserWarning: Expected 3 arguments for function <function process_files at 0x7b4cd9f8a3b0>, received 2.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1006: UserWarning: Expected at least 3 arguments for function <function process_files at 0x7b4cd9f8a3b0>, received 2.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://dcaab98a4675273cbf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
